In [ ]:
from pathlib import Path
import glob

# Finds the dataset folder that contains the checkpoints
CKPT_MATCH = list(Path("/kaggle/input").glob("**/best_attn1_freeze0.pt"))
if not CKPT_MATCH:
    raise FileNotFoundError("best_attn1_freeze0.pt not found under /kaggle/input")

CKPT_ROOT = CKPT_MATCH[0].parent
print("CKPT_ROOT =", CKPT_ROOT)
print("Files:", sorted([p.name for p in CKPT_ROOT.glob("*.pt")]))


In [ ]:
import torch

p = CKPT_ROOT / "best_attn1_freeze0.pt"
obj = torch.load(p, map_location="cpu")

print(type(obj))
if isinstance(obj, dict):
    print("dict keys (first 20):", list(obj.keys())[:20])
    # heuristic: state_dict usually has tensor values
    any_tensor = any(hasattr(v, "shape") for v in obj.values())
    print("looks_like_state_dict =", any_tensor)


# From Notebook-3:
- Conv blocks
- Self-attention (if used)
- ResNet encoder wrapper (if used)
- Your final model class

# Example placeholder name:
- class OursResUNet(nn.Module)


In [ ]:
import torch

def build_ours_from_ckpt(attn: int, freeze: int, ckpt_root: Path):
    # IMPORTANT: Use the same model init args as Notebook-3
    model = OursResUNet(use_attention=bool(attn), freeze_encoder=bool(freeze))  # <-- adjust to your signature

    ckpt_path = ckpt_root / f"best_attn{attn}_freeze{freeze}.pt"
    state = torch.load(ckpt_path, map_location="cpu")
    model.load_state_dict(state, strict=True)
    return model

# Quick check
_ = build_ours_from_ckpt(attn=1, freeze=0, ckpt_root=CKPT_ROOT)
print("Loaded Ours attn=1 freeze=0")


In [ ]:
def build_model(name):
    if name == "UNet":
        return UNet(in_ch=3, out_ch=1, base=32)
    if name == "ResUNet34":
        return ResUNet34(pretrained=True, out_ch=1)
    if name == "AttnUNet":
        return AttnUNet(in_ch=3, out_ch=1, base=32)

    # ---- Ours (from Notebook-3 checkpoint) ----
    if name == "Ours_AttnFinetune":
        return build_ours_from_ckpt(attn=1, freeze=0, ckpt_root=CKPT_ROOT)

    # Optional ablation variants:
    if name == "Ours_NoAttnFinetune":
        return build_ours_from_ckpt(attn=0, freeze=0, ckpt_root=CKPT_ROOT)
    if name == "Ours_NoAttnFrozen":
        return build_ours_from_ckpt(attn=0, freeze=1, ckpt_root=CKPT_ROOT)
    if name == "Ours_AttnFrozen":
        return build_ours_from_ckpt(attn=1, freeze=1, ckpt_root=CKPT_ROOT)

    raise ValueError("Unknown model: " + name)

MODEL_NAMES = ["UNet", "ResUNet34", "AttnUNet", "Ours_AttnFinetune"]


In [ ]:
is_ours = mname.startswith("Ours_")
if is_ours:
    model = build_model(mname).to(device)
    model.eval()
    # Tune threshold on fold-val and evaluate on fold-test (same as others)
    tuned = tune_threshold_on_val(model, val_loader, val_ref, CFG["thresholds"])
    best_t = tuned["threshold"]

    test_fixed = eval_slice_and_patient(model, test_loader, test_ref, threshold=0.5)
    test_tuned = eval_slice_and_patient(model, test_loader, test_ref, threshold=best_t)

    all_rows.append({
        "fold": fold_idx,
        "model": mname,
        "best_epoch_by_val_patient_dice@0.5": -1,
        "val_best_patient_dice@0.5": np.nan,
        "val_best_threshold": float(best_t),
        "test_patient_dice@0.5": float(test_fixed["patient_dice_mean"]),
        "test_patient_dice@tuned": float(test_tuned["patient_dice_mean"]),
        "test_slice_dice@0.5": float(test_fixed["slice_dice_mean"]),
        "test_slice_dice@tuned": float(test_tuned["slice_dice_mean"]),
    })
    continue
